In [1]:
# importing dependencies
import pandas as pd
import gmaps
import requests

# importing google api key
from config import g_key

# storing CSV into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")

city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hasaki,JP,2022-07-15 11:08:01,35.7333,140.8333,75.60,93,100,8.03
1,1,Avarua,CK,2022-07-15 11:08:02,-21.2078,-159.7750,73.45,68,100,8.05
2,2,Hobart,AU,2022-07-15 11:08:02,-42.8794,147.3294,46.26,83,40,8.05
3,3,Jalu,LY,2022-07-15 11:08:03,29.0331,21.5482,89.64,23,0,8.99
4,4,Chokurdakh,RU,2022-07-15 11:08:03,70.6333,147.9167,63.95,59,100,3.06


In [2]:
# gmaps needs either an integer or floating point
# Check the dtypes of the new dataframe

city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# creating a max temp heatmap

# tell gmaps to use API key, only needs to be done once

gmaps.configure(api_key=g_key)

In [4]:
# Assign locations to an arry of the latitude and longitude pairs

locations = city_data_df[["Lat", "Lng"]]

# Assign the weights variable to some values

max_temp = city_data_df["Max Temp"]

# Assign the figure variable

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300,
                                point_radius=4)

# Add the heatmap layer

fig.add_layer(heat_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

# Google Heatmaps and Negative Numbers

- Note that heatmaps cannot plot negative numbers
    - if you have a max temp that is < 0, you get a `InvalidWeightException` error 
        - for this line of code `heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)`
        
- You can remove negative temperatures with a `for` loop to iterate through `max_temp` and add the temps that are greater than 0 to a new list:

    `max_temp = city_data_df["Max Temp"]`

    `temps = []`

    `for temp in max_temp:`
        `temps.append(max(temp, 0))`

- Here you're using the max() function to get teh largest value between temp and 0. 
    - If the `temp` is less than 0, then 0 will be added to the list in its place.
    - If `temp` is greater than 0 then `temp` will be added to the list.
    
- You can accomplist the same with a list comprehension:

`heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp])`

# Adjusting the heatmap zoom, intensity, and point radius

- first add the geographic center of earth in the form of latitude and longitude 
    - 30.0° N and 31.0° E
    - set the center and zoom level on the `gmaps.figure()`

- Add zoom level so that only one map of earth is shown

- Need to change the circle so that they are larger to show the temperature gradient differences.
    - documentation notes there is a dissipation option for heatmaps that can be added to the `gmaps.heat_layer()` attirbute. The options are:
        - Default option is "True", here set it to "False"
        - can add `max_intensity` to make each measurement have a better gradient varience.
        - can add `point_radius` to make each measurement radius larger

In [5]:
# adding a heatmap for humidity 

locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# adding a heatmap for cloudiness

locations = city_data_df[["Lat", "Lng"]]

clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity= 300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Creating a wind speed heatmap

locations = city_data_df[["Lat", "Lng"]]

wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# ask the custom to add min and max tmep

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trp? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trp? 90


In [9]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hasaki,JP,2022-07-15 11:08:01,35.7333,140.8333,75.60,93,100,8.03
3,3,Jalu,LY,2022-07-15 11:08:03,29.0331,21.5482,89.64,23,0,8.99
7,7,Pochutla,MX,2022-07-15 11:08:04,15.7432,-96.4661,75.81,73,100,4.16
8,8,Georgetown,MY,2022-07-15 11:05:08,5.4112,100.3354,89.53,66,20,5.75
11,11,Thap Khlo,TH,2022-07-15 11:08:06,16.1600,100.5966,86.16,75,100,1.77
...,...,...,...,...,...,...,...,...,...,...
512,512,Morant Bay,JM,2022-07-15 11:22:30,17.8815,-76.4093,80.22,83,50,5.44
514,514,Loa Janan,ID,2022-07-15 11:22:31,-0.5830,117.0950,75.38,90,100,1.88
516,516,Vila Franca Do Campo,PT,2022-07-15 11:22:32,37.7167,-25.4333,75.61,88,73,12.21
524,524,Kati,ML,2022-07-15 11:22:35,12.7441,-8.0726,85.41,50,88,7.54


In [10]:
# determining if there are null vales in prefrred cities
# It is a good idea to keep the number of cities below 200 so that it's easier to plot markers on heatmap.
# If you have some rows with null values, you need to drop them using dropna() at the end of your filtering 
preferred_cities_df.count()

City_ID       180
City          180
Country       178
Date          180
Lat           180
Lng           180
Max Temp      180
Humidity      180
Cloudiness    180
Wind Speed    180
dtype: int64

# Map Vacation Criteria

- Now creating a heatmap for the max temperature for the filtered cites
- Also add a marker for each city that will display the following when the marker is clicked:
    - name of the city
    - country code
    - max temp, 
    - name of nearby hotel within 3 miles of the coordinates 
    
- To do this, use the coordinates in the preferred_cities_df and the Google Places API and retrieve hotel info

## Getting the Travel Destinations

- Will need to create a new DataFrame for the hotels because the preferred_cities_df is the filted DF that the customer will always filter.
    - Make a copy of preferred_cities_df and name it hotel_df
        - keep the same column names and add col. to hold name of hotel. 


In [11]:
# Create DataFrame called hotel_df to store hotel names with city, country, max temp, and coordinates

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hasaki,JP,75.60,35.7333,140.8333,
3,Jalu,LY,89.64,29.0331,21.5482,
7,Pochutla,MX,75.81,15.7432,-96.4661,
8,Georgetown,MY,89.53,5.4112,100.3354,
11,Thap Khlo,TH,86.16,16.1600,100.5966,
13,Quatre Cocos,MU,75.31,-20.2078,57.7625,
21,Inhambane,MZ,75.56,-23.8650,35.3833,
28,Atuona,PF,76.91,-9.8000,-139.0333,
29,Mufumbwe,ZM,79.68,-13.6833,24.8000,
30,Benguela,AO,76.26,-12.5763,13.4055,


# Retrieve Hotels from Nearby Search

- This will require an API request
- Need to set the parameters for the search

## Setting the Parameters for Nearby Search

- refer to the API Nearby Search documentation:
    - [click here for doc](https://developers.google.com/places/web-service/search#PlaceSearchRequests%0D%0A)
    
- Required Parameters: 
    - output type: JSON or XML
    - key: api key
    - location: expressed in latitude, longitude
    - radius: defines distance in meters within which to return place results
        - max allowed radius is 50,000 meters
        - note that radius must not be included if `rankby=distance` is specified
            - if using `rankby=distance`, then one or more of `keyword, name, or type` must be used.

- For this project our parameters are:
    - Your API key
    - Lat, Lng for location
    - radius=5000
    - type of place (found under the optional parameters)
        - cf the [Place Type Guide](https://developers.google.com/places/web-service/supported_types)
        
- The base URL for nearby search returning a json output:
    - https://maps.googleapis.com/maps/api/place/nearbysearch/json 
    - The Python Requests Library Doc states can add parameters to a URL through using a dictionary of strings
        - requires using the `params` keyword argument:
            - `requests.get("base URL", params=parameters)`
        - [Requests Library Doc](https://requests.kennethreitz.org/en/master/)
        
## Iterate Through the hotel_df DataFrame

- Can use the iterrows() function
    - need to provide index and the row in the for loop using this syntax:
        - `for index, row in df.itterrows()
    - iterrows() iterates over a DataFrame as Index, Series pairs
        cf the [iterrows() documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html)
        

In [14]:
# Set the parameters to search for a hotel

params = {
    "radius": 5000, 
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the lat and lng to the location key for the params dictionary
    
    params["location"] = f"{lat}, {lng}"
    
    # use search term "lodging" and our lat and lng
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search

    
    hotels = requests.get(base_url, params=params).json()
    
    # Grab first hotel from the results and store the name
    # using a try-except block to handle index error
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
        
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head(30)

info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Max Temp</dt><dd>{Max Temp}</dd>

</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# creating a heatmap of temp for cataion spots using max temp from hotel_df

locations = hotel_df[["Lat", "Lng" ]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# adding a heat layer for max temp
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)

# adding a marker layer for the hotel locations
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Adding an info block

- Clicking on a marker and getting desired information

- First initiate a variable `info_box_template` and assign to it a mult-line string using three quotes
    - Text inside quotes is **HTML Code** 
        - HTML is defined by opening and lcosing angular brackets (i.e. `<tag>` and `<tag/>`)
        - the angular brackets always come in pairs with the closing being preceded by "/"
        - In between brackets is text, called the tag
            - `<dl>` : description list
            - `<dt>` : term or name in dl that is nested under the `<dl>` tag
            - `<dd>` : used to define the term or name `<dt>` tag
        - Written on paper, HTML code would look like the following:
            - Description list: `<dl>~
                - Description term: `<dt>`
                    - Description definition: `<dd>`
                - Description term: `<dt>`
                    - Description definition: `<dd>`
- For this project, add the following from `hotel_df` as the description definition:
    - hotel name
    - city name
    - country code
    - max temp
    
- All of this will be set as the `info_block_template` variable